In [2]:
import sqlite3
import collections
import nltk
import gensim
import pymorphy2
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim import models
from gensim.models import LdaModel, LdaMulticore
from gensim.summarization import summarize, keywords

In [7]:
conn = sqlite3.connect('C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\updated_vacancies.db')
cursor = conn.cursor()

cursor.execute('SELECT * from vacancies')
result = cursor.fetchall()

conn.close()

In [8]:
result[:5]

[('https://hh.ru/vacancy/29470150',
  'Менеджер по продажам продуктов эквайринга',
  'Саранск',
  '2019-01-15',
  'Функции  Реализует спектр продуктов в рамках моделей продаж и четко определенных политик, стандартов качества   Обеспечивает выполнение плановых показателей по своим клиентам или перечню продуктов на краткосрочный период   Выстраивает отношения с клиентами в целях продажи дополнительных продуктов   Работает с жалобами и претензиями клиентов   Привлекает клиентов по своей специализации в соответствии с планом, указаниями руководства   Ведет документацию по продажам, организует или инициирует согласование документов с клиентами и внутри Банка   Взаимодействует со смежными подразделениями в рамках совместных сделок   Участвует в пилотировании продуктов и оптимизации процессов, предоставляя обратную связь   В ряде случаев, отвечает за постановку задач, обучение и контроль деятельности команды  Образование  Высшее в Экономика/бухгалтерия или Управление  Опыт работы  Опыт работы

In [9]:
test = result[1][4]
test = test + result[0][4]
test1 = nltk.sent_tokenize(test)

In [7]:
test1[:5]

['Если ты:  хочешь построить карьеру в банковском секторе; стремишься стать лидером и развиваться профессионально и личностно; нацелен на достижение лучших результатов для клиентов; мечтаешь работать в инновационной команде профессионалов.',
 'Начни карьеру в крупнейшем банке России и СНГ!',
 'Как консультант, ты будешь:  продавать онлайн-продукты и услуги банка клиентам в отделении, используя планшет и терминалы самообслуживания (Автоплатеж, Онлайн-вклады, Мобильный банк); объяснять клиентам преимущества использования устройств самообслуживания и проводить обучения по работе с ними; выявлять цели визита клиента в банк и направлять его к соответствующим специалистам отделения.',
 'Ты подходишь нам, если ты:  специалист со средним профессиональным/неполным высшим образованием; владеешь программами Microsoft Office, Internet Explorer, Outlook; выдерживаешь высокий темп работы; готов работать в операционном зале в течение всего рабочего дня; доброжелательно относишься к окружающим, готов 

In [10]:
def remove_stopwords(words):
    return [word for word in words if word not in stopwords.words('english')]

def prepare_text(text):
    return [remove_stopwords(gensim.utils.simple_preprocess(str(sentence), deacc=True)) \
               for sentence in text]

In [14]:
test1 = prepare_text(test1)
test1[0][:7]

['если', 'ты', 'хочешь', 'построить', 'карьеру', 'банковском', 'секторе']

In [15]:
sample_dictionary = corpora.Dictionary(test1)
print(sample_dictionary)
print(sample_dictionary.token2id)

Dictionary(301 unique tokens: ['банковском', 'для', 'достижение', 'если', 'инновационнои']...)
{'банковском': 0, 'для': 1, 'достижение': 2, 'если': 3, 'инновационнои': 4, 'карьеру': 5, 'клиентов': 6, 'команде': 7, 'лидером': 8, 'личностно': 9, 'лучших': 10, 'мечтаешь': 11, 'на': 12, 'нацелен': 13, 'построить': 14, 'профессионалов': 15, 'профессионально': 16, 'работать': 17, 'развиваться': 18, 'результатов': 19, 'секторе': 20, 'стать': 21, 'стремишься': 22, 'ты': 23, 'хочешь': 24, 'банке': 25, 'крупнеишем': 26, 'начни': 27, 'россии': 28, 'снг': 29, 'автоплатеж': 30, 'банк': 31, 'банка': 32, 'будешь': 33, 'визита': 34, 'вклады': 35, 'выявлять': 36, 'его': 37, 'использования': 38, 'используя': 39, 'как': 40, 'клиента': 41, 'клиентам': 42, 'консультант': 43, 'мобильныи': 44, 'направлять': 45, 'ними': 46, 'обучения': 47, 'объяснять': 48, 'онлаин': 49, 'отделении': 50, 'отделения': 51, 'планшет': 52, 'по': 53, 'преимущества': 54, 'проводить': 55, 'продавать': 56, 'продукты': 57, 'работе': 58

In [16]:
sample_text_corpus = [test]
sample_text_corpus = [simple_preprocess(sent) for sent in sample_text_corpus]

sample_dictionary_corpus = corpora.Dictionary(sample_text_corpus)
sample_corpus = [sample_dictionary_corpus.doc2bow(sent, allow_update=True) for sent in sample_text_corpus]
print(sample_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 2), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 5), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 3), (85, 2), (86, 4), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 2), (95, 2), (96, 1), (97, 2), (98, 2), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [17]:
def decipher(dictionary, corpus):
    return [[(dictionary[num], round(count, 2)) for num, count in line] for line in corpus]

In [18]:
print(decipher(sample_dictionary_corpus, sample_corpus))

[[('explorer', 1), ('internet', 1), ('microsoft', 1), ('office', 1), ('outlook', 1), ('автоплатеж', 1), ('анализ', 1), ('базовые', 1), ('базовый', 2), ('банк', 2), ('банка', 2), ('банке', 2), ('банковском', 1), ('будешь', 1), ('будущее', 1), ('бухгалтерия', 1), ('быстрее', 1), ('вакансия', 1), ('ведет', 1), ('взаимодействует', 1), ('видеоинтервью', 1), ('визита', 1), ('вклады', 1), ('владеешь', 1), ('вместе', 1), ('внутри', 1), ('возможность', 1), ('вознаграждение', 1), ('возникновении', 1), ('возраста', 1), ('время', 1), ('всего', 1), ('выбирает', 1), ('выдерживаешь', 1), ('выполнение', 1), ('высокие', 1), ('высокий', 1), ('выстраивает', 1), ('высшее', 1), ('высшим', 1), ('выявлять', 1), ('гарантированный', 1), ('года', 2), ('годовая', 1), ('готов', 2), ('деятельности', 1), ('для', 1), ('дмс', 2), ('дня', 1), ('добивается', 1), ('доброжелательно', 1), ('документацию', 1), ('документов', 1), ('домом', 1), ('дополнительных', 1), ('достижение', 1), ('достижения', 1), ('доход', 1), ('дохо

In [19]:
bigram = gensim.models.phrases.Phrases(test1, min_count=3, threshold=10)
sample_text_bigram = [bigram[sent] for sent in test1]
print(sample_text_bigram)

[['если', 'ты', 'хочешь', 'построить', 'карьеру', 'банковском', 'секторе', 'стремишься', 'стать', 'лидером', 'развиваться', 'профессионально', 'личностно', 'нацелен', 'на', 'достижение', 'лучших', 'результатов', 'для', 'клиентов', 'мечтаешь', 'работать', 'инновационнои', 'команде', 'профессионалов'], ['начни', 'карьеру', 'крупнеишем', 'банке', 'россии', 'снг'], ['как', 'консультант', 'ты', 'будешь', 'продавать', 'онлаин', 'продукты', 'услуги', 'банка', 'клиентам', 'отделении', 'используя', 'планшет', 'терминалы', 'автоплатеж', 'онлаин', 'вклады', 'мобильныи', 'банк', 'объяснять', 'клиентам', 'преимущества', 'использования', 'устроиств', 'проводить', 'обучения', 'по', 'работе', 'ними', 'выявлять', 'цели', 'визита', 'клиента', 'банк', 'направлять', 'его', 'соответствующим', 'специалистам', 'отделения'], ['ты', 'подходишь', 'нам', 'если', 'ты', 'специалист', 'со', 'средним', 'неполным', 'высшим', 'образованием', 'владеешь', 'программами', 'microsoft', 'office', 'internet', 'explorer', 'ou

In [20]:
task_text = test

In [22]:
sample_tfidf = models.TfidfModel(sample_corpus, smartirs='ntc')
print(decipher(sample_dictionary_corpus, sample_tfidf[sample_corpus]))

[[('explorer', 0.04), ('internet', 0.04), ('microsoft', 0.04), ('office', 0.04), ('outlook', 0.04), ('автоплатеж', 0.04), ('анализ', 0.04), ('базовые', 0.04), ('базовый', 0.08), ('банк', 0.08), ('банка', 0.08), ('банке', 0.08), ('банковском', 0.04), ('будешь', 0.04), ('будущее', 0.04), ('бухгалтерия', 0.04), ('быстрее', 0.04), ('вакансия', 0.04), ('ведет', 0.04), ('взаимодействует', 0.04), ('видеоинтервью', 0.04), ('визита', 0.04), ('вклады', 0.04), ('владеешь', 0.04), ('вместе', 0.04), ('внутри', 0.04), ('возможность', 0.04), ('вознаграждение', 0.04), ('возникновении', 0.04), ('возраста', 0.04), ('время', 0.04), ('всего', 0.04), ('выбирает', 0.04), ('выдерживаешь', 0.04), ('выполнение', 0.04), ('высокие', 0.04), ('высокий', 0.04), ('выстраивает', 0.04), ('высшее', 0.04), ('высшим', 0.04), ('выявлять', 0.04), ('гарантированный', 0.04), ('года', 0.08), ('годовая', 0.04), ('готов', 0.08), ('деятельности', 0.04), ('для', 0.04), ('дмс', 0.08), ('дня', 0.04), ('добивается', 0.04), ('доброже

In [23]:
LDA_dictionary = corpora.Dictionary(sample_text_bigram)
LDA_corpus = [LDA_dictionary.doc2bow(sent) for sent in sample_text_bigram]

LDA_model = LdaMulticore(corpus=LDA_corpus,
                         id2word=LDA_dictionary,
                         num_topics=1, #РєРѕР»РёС‡РµСЃС‚РІРѕ С‚РµРј
                         passes=10,
                         chunksize=5, # РєРѕР»РёС‡РµСЃС‚РІРѕ РїРѕРґСЃРїРёСЃРєРѕРІ РёСЃРїРѕР»СЊР·СѓРµРјС‹С… РІ РєР°Р¶РґРѕРј РїСЂРѕС…РѕРґРµ
                         iterations=100, 
                         gamma_threshold=0.001,
                         per_word_topics=True)
LDA_model.save('LDA_model.model')

print(LDA_model.print_topics())

just_text = test

summarize(just_text, word_count=20)
print(keywords(just_text))

[(0, '0.010*"продуктов" + 0.009*"по" + 0.009*"или" + 0.007*"работы" + 0.007*"ты" + 0.007*"клиентов" + 0.006*"рамках" + 0.006*"случаев" + 0.006*"работать" + 0.006*"опыт"')]
или
продуктов
клиентов
банка клиентам
случаев
онлаин
чтобы
руководства
корпоративную
корпоративная
трудоустроиство
работы готов
работать
помощь
продажи
навыки
обучение
года
льготные условия
кредитования
основные характеристики
премия
как
специалистам
статуса
используя
нам
уровень
самообслуживания
рамках
регулярное
заболевании
базовыи
жизнь
разного
выстраивает
период
возраста
выбирает что
удобного
секторе
обратную
процессов
деятельности
образование
стать
выявлять
визита
проиди
банке
всего
доброжелательно
